# Amazon CloudFront 캐싱

## CloudFront의 캐싱 티어
- 클라우드프론트의 캐싱은 2티어로 이루어짐
- 요청 순서
    1. Edge Location
    2. Regional Edge Cache
    3. Origin
    - 유저는 클라우드프론트를 통해 엣지 로케이션에 캐시를 요청, 엣지 로케이션에 캐시가 없으면 리저널 엣지 캐시에 요청, 여기에도 없으면 오리진

## 클라우드프론트의 Cache Key
- 요청에 따라 어떤 캐시 내용을 보여줄지를 결정하는 정보의 조합
    - 각 오브젝트는 고유의 캐시 키 단위로 캐시됨
- Cache Hit : 뷰어가 특정 캐시 키로 오브젝트를 요청하였을 때, 엣지 로케이션에서 해당 캐시 오브젝트를 가지고 이어 원본에 요청 과정 없이 제공할 수 있는 상황
    - Origin의 부하 경감 가능
    - 더 빠르게 컨텐츠 제공 가능
    - 즉 CDN이 캐시 힛이 많을수록 더 좋은 퍼포먼스 제공
- 주요 캐시 키 구성 요소
    - 경로, Query String, HTTP Header, Cookie

### HTTP Header Based Cache
- Cache Key중 HTTP Header를 활용
- 활용 사례
    - 언어별 캐싱
    - Device Type 별로 캐싱
        - CloudFront에서 별도로 User-Agent를 기반으로 전용 헤더 생성
        - CloudFront-Is-Desktop_Viewer, CloudFront-Is-Mobile_Viewer, CloudFront-Is-SmartTV-Viewer 등
    - 지역별 캐싱
        - CloudFront에서 전용 헤더 생성 : CloudFront-Viewer-Country
- 헤더 명은 대소문자를 구분하지 않지만, 값은 구분

### Cookie Based Cache
- Cookie를 기반으로 컨텐츠 내용을 캐시
    - 쿠키를 활용하지 않는 HTTP서버 혹은 S3에서 사용할 경우 퍼포먼스만 저하될 수 있음

## 캐시 만료
- 캐시된 오브젝트는 일정 기간(TTL : Time To Live) 이후 만료
    - 그 다음 요청이 올 경우 CloudFront는 Origin에 오브젝트 갱신 여부를 확인
    - Origin이 304 Not Modified를 줄 경우 갱신 필요 없음
    - 200 ok와 파일을 줄 경우에는 갱신

## 캐시 TTL
- 캐시 오브젝트를 얼마나 오래 보관할지에 관한 설정
    - 기본 24시간
    - 모든 클라우드프론트의 오브젝트에 적용
    - 파일 단위에서는 오리진에서 Cache-Control Header 혹은 Expires Header를 포함하여 조절 가능(클라우드 프론트의 헤더보다 우선 적용, 하지만 클라우드 프론트에서 설정한 Min/Max TTL 범위를 벗어나는 것은 불가능)
- TTL의 종류
    - Minimum TTL : 최소 TTL, 파일 단위 컨트롤에서 줄 수 있는 최소 TTL
    - Maximum TTL : 최대 TTL, 파일 단위 컨트롤에서 줄 수 있는 최대 TTL
    - Default TTL : 별도의 설정이 없을 경우 부여되는 기본 TTL

### 캐시 TTL Control
- 파일 단위에서는 오리진에서 Cache-Control Header 혹은 Expires Header를 포함해서 조절 가능
- 캐시 컨트롤 : 얼마나 오래 오브젝트를 캐시하는지 기간 설정
    - max-age : CloudFront와 브라우저 둘 다 영향
    - s-maxage : CloudFront만 영향
    - no-cache, no-store : 캐싱하지 않음(단, Min TTL이 0 이상인 경우 Min TTL로 최저 설정)
- Expires : Cache가 만료되는 정확한 시간을 설정
    - 클라우드프론트와 브라우저 영향
- CloudFront의 Min/Max TTL 범위 안에서만 설정 가능

### 캐시 정책
- 캐싱과 관련된 내용을 정책으로 정의하여 클라우드프론트에 적용 가능
- 주요 설정
    - 어떤 키(HTTP Header, 쿠키, Query String등)로 컨텐츠를 캐시하는지
    - 얼마나 오래 캐시하는지 : TTL
    - 컨텐츠 압축 저장 관련 설정
- 두 가지 종류
    - Managed : AWS에서 직접 생성한 정책으로 다양한 상황을 위해 미리 준비된 정책
    - 커스텀 : 직접 설정